<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


## Practice Project - Create a machine learning pipeline for a regression project


Estimated time needed: **90** minutes


## Scenario


You are a data engineer at a data analytics consulting company. Your company prides itself in being able to efficiently handle huge datasets. Data scientists in your office need to work with different algorithms and data in different formats. While they are good at Machine Learning, they count on you to be able do ETL jobs and build ML pipelines.


## Objectives

In this 4 part assignment you will:

- Part 1 ETL
  - Load a csv dataset
  - Remove duplicates if any
  - Drop rows with null values if any
  - Make transformations
  - Store the cleaned data in parquet format
- Part 2 Machine Learning Pipeline creation
  - Create a machine learning pipeline for prediction
- Part 3 Model evaluation
  - Evaluate the model using metrics
  - Print the intercept and the coefficients
- Part 4 Model Persistance
  - Cave the model for future production use
  - Load and verify the stored model


## Datasets

In this lab you will be using dataset(s):

- Modified version of car mileage dataset. Original dataset available at https://archive.ics.uci.edu/ml/datasets/auto+mpg


---


## Setup


For this lab, we will be using the following libraries:

- [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to
connect to this cluster.


The following required libraries are **not** pre-installed in the Skills Network Labs environment. **You will need to run the following cell** to install them:


In [2]:
%pip install pyspark
%pip install findspark

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [3]:
# You can also use this section to suppress warnings generated by your code:
import warnings
import findspark


def warn(*args, **kwargs):
    pass


warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

findspark.init()

## Part 1 - ETL


### Task 1 - Import required libraries


In [5]:
# your code goes here
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

### Task 2 - Create a spark session


In [6]:
# Create SparkSession

spark = SparkSession.builder\
    .appName('LinearRegression Practice Project')\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/12/08 09:17:21 WARN Utils: Your hostname, fcode resolves to a loopback address: 127.0.1.1; using 172.27.243.144 instead (on interface eth0)
23/12/08 09:17:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/08 09:17:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Task 3 - Load the csv file into a dataframe


Download the data file


In [7]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv


--2023-12-08 09:17:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 198.23.119.253, 169.45.118.97, ...
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14354 (14K) [text/csv]
Saving to: ‘mpg-raw.csv’

mpg-raw.csv         100%[===================>]  14.02K  --.-KB/s    in 0s      

2023-12-08 09:17:49 (265 MB/s) - ‘mpg-raw.csv’ saved [14354/14354]



Load the dataset into the spark dataframe


In [23]:
# Load dataset
df = spark.read.csv('mpg-raw.csv', header=True, inferSchema=True)

### Task 4 - Print top 5 rows of the dataset


In [24]:
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|46.6|        4|       86.0|        65|  2110|      17.9|  80|Japanese|
|44.6|        4|       91.0|        67|  1850|      13.8|  80|Japanese|
|44.3|        4|       90.0|        48|  2085|      21.7|  80|European|
|44.0|        4|       97.0|        52|  2130|      24.6|  82|European|
|43.4|        4|       90.0|        48|  2335|      23.7|  80|European|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



In [25]:
df.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



### Task 5 - Print the number of cars in each Origin


In [26]:
df.groupBy('Origin').count().orderBy('count').show()

+--------+-----+
|  Origin|count|
+--------+-----+
|    NULL|    1|
|European|   70|
|Japanese|   88|
|American|  247|
+--------+-----+



### Task 6 - Print the total number of rows in the dataset


In [27]:
# your code goes here
rowcount1 = df.count()
print(rowcount1)

406


### Task 7 - Drop all the duplicate rows from the dataset


In [28]:
df = df.dropDuplicates()

### Task 8 - Print the total number of rows in the dataset


In [29]:
rowcount2 = df.count()
print(rowcount2)

392


### Task 9 - Drop all the rows that contain null values from the dataset


In [30]:
df = df.dropna()

### Task 10 - Print the total number of rows in the dataset


In [31]:
# your code goes here
rowcount3 = df.count()
print(rowcount3)

385


### Task 11 - Rename the column "Engine Disp" to "Engine_Disp"Drop


In [32]:
# your code goes here

df = df.withColumnRenamed('Engine Disp', 'Engine_Disp')

In [33]:
df.show(3)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 3 rows



### Task 12 - Save the dataframe in parquet format, name the file as "mpg-cleaned.parquet"


In [34]:
cleaned_saved_file = 'mpg-cleaned.parquet'

df.write.mode('overwrite').parquet(cleaned_saved_file)

#### Part 1 - Evaluation


Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [35]:
import os
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("Renamed column name = ", df.columns[2])


print("mpg-cleaned.parquet exists :", os.path.isdir("mpg-cleaned.parquet"))

Part 1 - Evaluation
Total rows =  406
Total rows after dropping duplicate rows =  392
Total rows after dropping duplicate rows and rows with null values =  385
Renamed column name =  Engine_Disp
mpg-cleaned.parquet exists : True


## Part - 2 Machine Learning Pipeline creation


### Task 1 - Load data from "mpg-cleaned.parquet" into a dataframe


In [36]:
# your code goes here

df = spark.read.parquet(cleaned_saved_file)
rowcount4 = df.count()

In [37]:
# show top 5 rows
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
|22.4|        6|      231.0|       110|  3415|      15.8|  81|American|
|20.5|        6|      231.0|       105|  3425|      16.9|  77|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



In [39]:
# print the schema of the dataframe
df.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine_Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



### Task 2 - Define the StringIndexer pipeline stage


In [41]:
# Stage - 1 Using StringIndexer convert the string column "Origin" into "OriginIndex"
indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")

### Task 3 - Define the VectorAssembler pipeline stage


In [42]:
# Stage 2 - assemble the input columns 'Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year' into a single column "features"

features_cols = ['Cylinders', 'Engine_Disp',
                 'Horsepower', 'Weight', 'Accelerate', 'Year']

assembler = VectorAssembler(inputCols=features_cols, outputCol='features')

### Task 4 - Define the StandardScaler pipeline stage


In [43]:
# Stage 3 - scale the "features" using standard scaler and store in "scaledFeatures" column
scaler = StandardScaler(
    inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=True)

<details>
    <summary>Click here for a Hint</summary>
    
Use the StandardScaler class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

```

</details>


### Task 5 - Define the StandardScaler pipeline stage


In [44]:
# Stage 4 - Create a LinearRegression stage to predict "MPG"

lr = LinearRegression(featuresCol='scaledFeatures', labelCol='MPG')

<details>
    <summary>Click here for a Hint</summary>
    
Use the LinearRegression class

</details>


<details>
    <summary>Click here for Solution</summary>

```python
lr = LinearRegression(featuresCol="features", labelCol="MPG")
```

</details>


### Task 6 - Build the pipeline


In [45]:
# Build a pipeline using the above four stages

pipeline = Pipeline(stages=[indexer, assembler, scaler, lr])

### Task 7 - Split the data


In [46]:
# Split the data into training and testing sets with 70:30 split. Use 42 as seed
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

### Task 8 - Fit the pipeline


In [48]:
# Fit the pipeline using the training data

pipelineModel = pipeline.fit(trainingData)

23/12/08 09:53:36 WARN Instrumentation: [884cb3e7] regParam is zero, which might cause numerical instability and overfitting.


#### Part 2 - Evaluation


Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [50]:
pipeline.getStages()

[StringIndexer_5e1698e7a2a6,
 VectorAssembler_25eb085d5c34,
 StandardScaler_24928472390c,
 LinearRegression_a80fa64dd258]

In [49]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  385
Pipeline Stage 1 =  StringIndexer
Pipeline Stage 2 =  VectorAssembler
Pipeline Stage 3 =  StandardScaler
Label column =  MPG


## Part 3 - Model Evaluation


### Task 1 - Predict using the model


In [51]:
# Make predictions on testing data
predictions = pipelineModel.transform(testingData)

### Task 2 - Print the MSE


In [52]:
# Your code goes here

evaluator = RegressionEvaluator(
    labelCol='MPG', predictionCol='prediction', metricName='mse')

mse = evaluator.evaluate(predictions)

print(mse)

12.226745835567348


### Task 3 - Print the MAE


In [53]:
# Your code goes here

evaluator = RegressionEvaluator(
    predictionCol='prediction', labelCol='MPG', metricName='mae')

mae = evaluator.evaluate(predictions)

print(mae)

2.8457151130131377


### Task 4 - Print the R-Squared(R2)


In [54]:
# Your code goes here

evaluator = RegressionEvaluator(
    predictionCol='prediction', labelCol='MPG', metricName='r2')

r2 = evaluator.evaluate(predictions)

print(r2)

0.8018737394896356


#### Part 3 - Evaluation


Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [55]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse, 2))
print("Mean Absolute Error = ", round(mae, 2))
print("R Squared = ", round(r2, 2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept, 2))

Part 3 - Evaluation
Mean Squared Error =  12.23
Mean Absolute Error =  2.85
R Squared =  0.8
Intercept =  23.83


## Part 4 - Model persistance


### Task 1 - Save the model to the path "Practice_Project"


In [57]:
# Save the pipeline model
pipelineModel.write().overwrite().save('Practice_Project')

### Task 2 - Load the model from the path "Practice_Project"


In [58]:
# Load the pipeline model
loadedPipelineModel = PipelineModel.load('Practice_Project')

### Task 3 - Make predictions using the loaded model on the test data


In [59]:
# Use the loaded pipeline model for predictions
predictions = loadedPipelineModel.transform(testingData)

### Task 4 - Show the predictions


In [60]:
# your code goes here
predictions.select('MPG', 'prediction').show(5)

+----+------------------+
| MPG|        prediction|
+----+------------------+
|10.0| 6.960764577508822|
|11.0| 8.545911819810659|
|12.0|10.226709705747316|
|12.0| 5.446415257215239|
|13.0| 21.43021240058852|
+----+------------------+
only showing top 5 rows



#### Part 4 - Evaluation


Run the code cell below.<br>
If the code throws up any errors, go back and review the code you have written.


In [61]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[1].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i, j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

Part 4 - Evaluation
Number of stages in the pipeline =  4
Coefficient for Cylinders is 0.119
Coefficient for Engine_Disp is 0.4971
Coefficient for Horsepower is -0.2517
Coefficient for Weight is -5.7923
Coefficient for Accelerate is 0.2369
Coefficient for Year is 2.9258


### Task 5 - Stop Spark Session


In [ ]:
spark.stop()

Congratulations!! you have successfully finished the practice project.


## Authors


[Ramesh Sannareddy](https://www.linkedin.com/in/rsannareddy/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMBD0231ENSkillsNetwork866-2023-01-01)


### Other Contributors


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description      |
| ----------------- | ------- | ----------------- | ----------------------- |
| 2023-05-26        | 0.1     | Ramesh Sannareddy | Initial Version Created |


Copyright © 2023 IBM Corporation. All rights reserved.
